<a href="https://colab.research.google.com/github/parshvak26/Text-to-SQL/blob/main/Text_to_SQL_DeepSeek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class DeepseekTxt2SQLPipeline:
    def __init__(self, model_folder: str,
                 system_message: str = None,
                 device: str = "cuda",
                 torch_dtype="auto"):
        """
        Initializes the Deepseek pipeline specialized for text-to-SQL tasks.

        Args:
            model_folder (str): Path to the model folder.
            system_message (str, optional): Custom system instruction message.
                Defaults to an SQL assistant instruction.
            device (str): The device to use ("cuda" by default).
            torch_dtype: Data type for model weights.
        """
        self.device = device

        # Clear memory before loading the model, conditionally for CUDA
        if self.device == "cuda":
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.empty_cache()  # Clear unused cached memory
            torch.cuda.ipc_collect()  # Collect unused memory
        else:
             gc.collect() # Collect garbage for CPU


        self.model = AutoModelForCausalLM.from_pretrained(
            model_folder,
            torch_dtype=torch_dtype,
            device_map=self.device
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_folder)

        # Default system message tailored for SQL query generation
        if system_message is None:
            self.system_message = (
                "You are Qwen, an expert SQL assistant. "
                "You help users convert natural language descriptions into efficient and accurate SQL queries."
            )
        else:
            self.system_message = system_message

    def apply_chat_template(self, messages, tokenize=False, add_generation_prompt=True):
        """
        Applies the chat template using the tokenizer's built-in method.
        Assumes that the tokenizer supports apply_chat_template.
        """
        return self.tokenizer.apply_chat_template(
            messages,
            tokenize=tokenize,
            add_generation_prompt=add_generation_prompt
        )

    def generate_response(self, prompt: str, max_new_tokens: int = 2000) -> str:
        """
        Generates a response using the Deepseek model.

        Args:
            prompt (str): The input prompt.
            max_new_tokens (int): Maximum number of tokens to generate.

        Returns:
            str: The model's generated response.
        """
        messages = [
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": prompt}
        ]

        # Build the full text prompt using the chat template
        text = self.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        # Tokenize the input text and move it to the proper device
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

        # Generate tokens from the model
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens
        )

        # Remove the input tokens from the generated output tokens
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        # Decode and return the generated text
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

    def generate_query(self, prompt: str, max_new_tokens: int = 2000) -> str:
        """
        Generates a SQL query based on a natural language description.

        Args:
            prompt (str): A description of the SQL query requirements.
            max_new_tokens (int): Maximum number of tokens for generation.

        Returns:
            str: The generated SQL query.
        """
        sql_prompt = (
            f"Convert the following natural language description into a SQL query:\n\n"
            f"{prompt}\n\nSQL Query:"
        )
        return self.generate_response(sql_prompt, max_new_tokens)

    def optimize_query(self, query: str,
                       optimization_instructions: str = "Optimize the following SQL query for better performance and readability.",
                       max_new_tokens: int = 2000) -> str:
        """
        Optimizes an existing SQL query.

        Args:
            query (str): The SQL query that needs optimization.
            optimization_instructions (str): Instructions for optimization.
            max_new_tokens (int): Maximum number of tokens for generation.

        Returns:
            str: The optimized SQL query.
        """
        prompt = f"{optimization_instructions}\n\n{query}\n\nOptimized SQL Query:"
        return self.generate_response(prompt, max_new_tokens)

    def generate_query_with_explanation(self, prompt: str, max_new_tokens: int = 2000) -> str:
        """
        Generates a SQL query and provides detailed explanations.

        Args:
            prompt (str): A description of the SQL query requirements.
            max_new_tokens (int): Maximum number of tokens for generation.

        Returns:
            str: The generated SQL query along with detailed explanations.
        """
        full_prompt = (
            f"Please generate a SQL query based on the following requirements: {prompt}\n\n"
            "After generating the SQL query, provide a detailed explanation of each part of the query."
        )
        return self.generate_response(full_prompt, max_new_tokens)

    def generate_diverse_drafts(self, prompt: str, max_new_tokens: int = 2000) -> dict:
        """
        Generates diverse SQL drafts in different dialects (Standard SQL, SQL Server, MySQL)
        and returns responses in three formats: text, markdown, and json.

        Args:
            prompt (str): The natural language description for the SQL query.
            max_new_tokens (int): Maximum number of tokens for generation.

        Returns:
            dict: A dictionary with keys 'text', 'markdown', and 'json' containing the responses.
        """
        # Create tailored prompts for each SQL dialect
        standard_prompt = f"Convert the following description into a Standard SQL query:\n{prompt}\nSQL Query:"
        sqlserver_prompt = f"Convert the following description into a SQL Server query:\n{prompt}\nSQL Query:"
        mysql_prompt = f"Convert the following description into a MySQL query:\n{prompt}\nSQL Query:"

        # Generate responses using the generate_response method
        standard_sql = self.generate_response(standard_prompt, max_new_tokens)
        sqlserver_sql = self.generate_response(sqlserver_prompt, max_new_tokens)
        mysql_sql = self.generate_response(mysql_prompt, max_new_tokens)

        # Compose plain text response
        text_response = (
            "Draft 1 (Standard SQL):\n" + standard_sql + "\n\n" +
            "Draft 2 (SQL Server):\n" + sqlserver_sql + "\n\n" +
            "Draft 3 (MySQL):\n" + mysql_sql
        )

        # Compose markdown response
        markdown_response = (
            "#### Draft 1: Standard SQL\n"
            "```sql\n" + standard_sql + "\n```\n\n"
            "#### Draft 2: SQL Server\n"
            "```sql\n" + sqlserver_sql + "\n```\n\n"
            "#### Draft 3: MySQL\n"
            "```sql\n" + mysql_sql + "\n```\n"
        )

        # Compose JSON response as a dictionary
        json_response = {
            "drafts": [
                {"dialect": "Standard SQL", "query": standard_sql},
                {"dialect": "SQL Server", "query": sqlserver_sql},
                {"dialect": "MySQL", "query": mysql_sql}
            ]
        }

        # Return responses in all formats
        return {
            "text": text_response,
            "markdown": markdown_response,
            "json": json_response
        }

In [4]:
# @title
import kagglehub

# Download latest version
path = kagglehub.model_download("deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b")

print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2


In [ ]:
# Specify the path to your model folder
model_folder = "/root/.cache/kagglehub/models/deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b/2"

# Initialize the pipeline
pipeline = DeepseekTxt2SQLPipeline(model_folder=model_folder, device="cpu")

# Example natural language prompt for SQL query generation
prompt = "Retrieve all orders placed in the last 30 days from the orders table."

# Generate diverse SQL drafts in multiple formats (plain text, markdown, and JSON)
diverse_drafts = pipeline.generate_diverse_drafts(prompt=prompt)

# Print the diverse outputs

# Plain Text Output
print("Plain Text Output:\n")
print(diverse_drafts['text'])
print("\n" + "-"*50 + "\n")

# Markdown Output
print("Markdown Output:\n")
print(diverse_drafts['markdown'])
print("\n" + "-"*50 + "\n")

# JSON Output
print("JSON Output:\n")
print(diverse_drafts['json'])

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
# 1) Install dependencies (quietly)
!pip -q install --upgrade "transformers>=4.44.0" "accelerate>=0.34.0" bitsandbytes sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 7.1 MB/s eta 0:00:00
